#Visualización de Variables Meteorológicas Mediante Sensores Remotos
Objetivos del proyecto:
Visualizar distintas variables meteorológicas dentro de AOIs previamente definidos y cargados en nuestra cuenta de GEE.

Documentación de sensores utilizados:

* ECMWF (European Centre for Medium-Range Wheather Forecast)  ERA5 por hora: https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands

**No usamos el DAILY porque hay información hasta el año 2020.**




## Importamos librería y nos logueamos en GEE

In [1]:
#Importación de librerías
import ee

#Nos autenticamos en Google Earth Engine
!earthengine authenticate
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=nIOUS_ZL698OaCaQMNLfr0xhljrjRgaoR6VCWMowGgY&tc=Ptqiyo9fJpck72G7iq4lJGE6d0HhvOk4WjYFyqKfl4A&cc=HPxKXTYFYdEg-Fb9u0xJXM6s0HYBexRFXkv3hI8_vSA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk67CLVQeKZOaE6Q9oX03WUa815bqxp3omS4I5k3aLZc4b0VXJhF9Wk

Successfully saved authorization token.


In [2]:
#Nos logueamos a la cuenta de Google Drive
from google.colab import drive    # La usamos para montar nuestra unidad de Google Drive
drive.mount('/content/drive')     # Montamos nuestra unidad de Google Drive

Mounted at /content/drive


In [3]:
#Instalamos e importamos GEEmap para visualizar la información geográfica en un mapa
!pip install geemap
import geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.6 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=8be1cd56138c22772be81db485559126c6d294ca352eeb31c7383

## Abrimos un mapa satelital de base

In [4]:
mapa = geemap.Map()

#Seteamos la vista de fondo en "satelital"
mapa.setOptions('SATELLITE')

mapa

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Cargamos los shape (AOIs) a analizar

In [5]:
#Cargamos el shape de la region BCP luego de almacenarlo en la cuenta de GEE
area_bcp = ee.FeatureCollection('users/jjablasco/BCP/area_bcp')
mapa.addLayer(area_bcp, {'color': 'red'})

In [ ]:
#Cargamos el shape de la zona norte luego de almacenarlo en la cuenta de GEE
norte = ee.FeatureCollection('users/jjablasco/BCP/norte')
mapa.addLayer(norte, {'color': 'purple'})

In [ ]:
#Cargamos el shape de la zona centro luego de almacenarlo en la cuenta de GEE
centro = ee.FeatureCollection('users/jjablasco/BCP/centro')
mapa.addLayer(centro, {'color': 'green'})

In [ ]:
#Cargamos el shape de la sur norte luego de almacenarlo en la cuenta de GEE
sur = ee.FeatureCollection('users/jjablasco/BCP/sur')
mapa.addLayer(sur, {'color': 'yellow'})

In [ ]:
#Shape de prueba del partido de Trenque Lauquen
trenque = ee.FeatureCollection('users/jjablasco/BCP/trenque')
mapa.addLayer(trenque, {'color': 'orange'})

### Definimos el AOI con el que trabajaremos y lo visualizamos

In [6]:
aoi = area_bcp #modificamos en función de lo necesario

#Hacemos zoom sobre la capa shape previamente cargada
mapa.centerObject(aoi, 6)   #cambiamos la escala de acuerdo al AOI
mapa

Map(bottom=754.0, center=[-37.30624194547664, -62.52767579673791], controls=(ZoomControl(options=['position', …

##TEMPERATURA

###Temperatura mínima del aire a 2 metros

Importamos las imágenes ECMWF ERA5 por hora

In [7]:
#Importamos la colección ERA5_Land agrupado por hora
ERA5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').select('temperature_2m')

In [8]:
#Filtrar desde el mes de octubre hasta marzo incluido para el ciclo de cultivo
fecha_inicio = '2023-02-18'
fecha_final = '2023-02-19'
temp_min = ERA5.filter(ee.Filter.date(fecha_inicio, fecha_final)).min().clip(aoi).subtract(273)

#### Visualizar temperatura mínima

In [9]:
#Definimos una paleta de colores HEX para el mapa
vis_params = {'min': 0, 'max': 12, 'palette': ['27a4f2','3eaef4','6ec2f7','9fd7f9','cfebfc']} #mas oscuro es menor temperatura

#Añadimos la capa de visualización a GEE
mapa.addLayer(temp_min, vis_params, 'Temperatura mínima ERA5')
mapa

Map(bottom=10324.0, center=[-37.30624194547664, -62.52767579673791], controls=(ZoomControl(options=['position'…

###Temperatura máxima del aire a 2 metros

Importamos las imágenes ECMWF ERA5 por hora

In [ ]:
#Importamos la colección ERA5_Land agrupado por hora
ERA5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').select('temperature_2m')

In [ ]:
#Filtrar desde el mes de octubre hasta marzo incluido para el ciclo de cultivo
fecha_inicio = '2023-02-18'
fecha_final = '2023-02-19'
temp_max = ERA5.filter(ee.Filter.date(fecha_inicio, fecha_final)).max().clip(aoi).subtract(273)

#### Visualizar temperatura máxima

In [ ]:
#Definimos una paleta de colores HEX para el mapa
vis_params = {'min': 10, 'max': 25, 'palette': ['ffa5a3','f57673','d34b47','b62a26','8f100d']} #mas oscuro es mayor temperatura

#Añadimos la capa de visualización a GEE
mapa.addLayer(temp_max, vis_params, 'Temperatura máxima ERA5')
mapa

Map(bottom=10410.0, center=[-38.788345355085625, -60.66100810627774], controls=(ZoomControl(options=['position…

Exportamos a nuestro Drive

In [ ]:
exportar_mapa = ee.batch.Export.image.toDrive(
  image= temp_min,   #modificar de acuerdo a la layer y AOI que querramos exportar
  description= 'Temperatura minima del aire',
  scale= 10,
  region= aoi.geometry(),
  maxPixels= 1e13,
  crs='EPSG:32720')

exportar_mapa.start()

In [ ]:
print('Task Status:', exportar_mapa.status()['state']) # Chequeamos el estado de la exportacion al drive

Task Status: FAILED


##HUMEDAD DEL SUELO

### Humedad en el suelo (0-7 cm de prof) como fracción (máx. 1)

Importamos las imágenes ECMWF ERA5 por hora

In [ ]:
#Importamos la colección ERA5_Land agrupado por hora
ERA5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').select('volumetric_soil_water_layer_1')

In [ ]:
#Filtrar el periodo de tiempo de interés
fecha_inicio = '2023-01-01'
fecha_final = '2023-01-02'
hum_suelo_0a7cm = ERA5.filter(ee.Filter.date(fecha_inicio, fecha_final)).median().clip(aoi)

#### Visualizar la humedad de suelo hasta 7 cm de prof.

In [ ]:
#Definimos una paleta de colores para el mapa
vis_params = {'min': 0, 'max': 0.35, 'palette': ['4d1103','992206','e43309','e4a20c','c8c8c8','c1ff89','5bff18','379b0e','13580b']}

#Añadimos la capa de visualización a GEE
mapa.addLayer(hum_suelo_0a7cm, vis_params, 'Humedad del suelo de 0 - 7 cm de prof. (%)')
mapa

Map(bottom=10374.0, center=[-38.169114135560854, -61.36962890625001], controls=(ZoomControl(options=['position…

Exportamos a nuestro Drive

In [ ]:
exportar_mapa = ee.batch.Export.image.toDrive(
  image= hum_suelo_0a7cm,   #modificar de acuerdo a la layer y AOI que querramos exportar
  description= 'Humedad superficial del suelo',
  scale= 10,
  region= aoi.geometry(),
  maxPixels= 1e13,
  crs='EPSG:32720')

exportar_mapa.start()

In [ ]:
print('Task Status:', exportar_mapa.status()['state']) # Chequeamos el estado de la exportacion al drive

### Humedad en el suelo (7-28 cm de prof) como fracción (máx 1)

Importamos las imágenes ECMWF ERA5 por hora

In [ ]:
#Importamos la colección ERA5_Land agrupado por hora
ERA5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').select('volumetric_soil_water_layer_2')

In [ ]:
#Filtrar el periodo de tiempo de interés
fecha_inicio = '2023-02-27'
fecha_final = '2023-02-28'
hum_suelo_7a28cm = ERA5.filter(ee.Filter.date(fecha_inicio, fecha_final)).median().clip(aoi)

#### Visualizar la humedad de suelo de 7-28 cm de prof.

In [ ]:
#Definimos una paleta de colores para el mapa
vis_params = {'min': 0, 'max': 0.35, 'palette': ['4d1103','992206','e43309','e4a20c','c8c8c8','c1ff89','5bff18','379b0e','13580b']}

#Añadimos la capa de visualización a GEE
mapa.addLayer(hum_suelo_7a28cm, vis_params, 'Humedad del suelo de 7 - 28 cm de prof. (%)')
mapa

Map(bottom=10374.0, center=[-38.169114135560854, -61.36962890625001], controls=(ZoomControl(options=['position…

Exportamos a nuestro Drive

In [ ]:
exportar_mapa = ee.batch.Export.image.toDrive(
  image= hum_suelo_7a28cm,   #modificar de acuerdo a la layer y AOI que querramos exportar
  description= 'Humedad subsuperficial del suelo',
  scale= 10,
  region= aoi.geometry(),
  maxPixels= 1e13,
  crs='EPSG:32720')

exportar_mapa.start()

In [ ]:
print('Task Status:', exportar_mapa.status()['state']) # Chequeamos el estado de la exportacion al drive

### Humedad en el suelo (0 - 28 cm de prof) como fracción (máx. 1)

Importamos las imágenes ECMWF ERA5 por hora

In [ ]:
#Importamos la colección ERA5_Land agrupado por hora
fecha_inicio = '2023-02-27'
fecha_final = '2023-02-28'

ERA5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filter(ee.Filter.date(fecha_inicio, fecha_final)).mean().clip(aoi)

In [ ]:
#Calculo humedad agregado
hum_suelo_0a28 = ERA5.expression('hum_suelo_0a7 * 0.25 + hum_suelo_7a28 * 0.75', {
    'hum_suelo_0a7' : ERA5.select('volumetric_soil_water_layer_1'),
    'hum_suelo_7a28' : ERA5.select('volumetric_soil_water_layer_2')
})

#### Visualizar la humedad de suelo de 0-28 cm de prof.

In [ ]:
#Definimos una paleta de colores para el mapa
vis_params = {'min': 0, 'max': 0.35, 'palette': ['4d1103','992206','e43309','e4a20c','c8c8c8','c1ff89','5bff18','379b0e','13580b']}

#Añadimos la capa de visualización a GEE
mapa.addLayer(hum_suelo_0a28, vis_params, 'Humedad del suelo de 0 - 28 cm de prof. (%)')
mapa

Map(bottom=10324.0, center=[-37.30624194547664, -62.52767579673791], controls=(ZoomControl(options=['position'…

Exportamos a nuestro Drive

In [ ]:
exportar_mapa = ee.batch.Export.image.toDrive(
  image= hum_suelo_0a28,   #modificar de acuerdo a la layer y AOI que querramos exportar
  description= 'Humedad subsuperficial del suelo',
  scale= 10,
  region= aoi.geometry(),
  maxPixels= 1e13,
  crs='EPSG:32720')

exportar_mapa.start()

In [ ]:
print('Task Status:', exportar_mapa.status()['state']) # Chequeamos el estado de la exportacion al drive

### Humedad en el suelo (0 - 100 cm de prof) como fracción (máx. 1)

Importamos las imágenes ECMWF ERA5 por hora

In [ ]:
#Importamos la colección ERA5_Land agrupado por hora
fecha_inicio = '2023-01-01'
fecha_final = '2023-01-02'

ERA5 = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filter(ee.Filter.date(fecha_inicio, fecha_final)).max().clip(aoi)

In [ ]:
#Calculo humedad agregado
hum_suelo_0a100 = ERA5.expression('hum_suelo_0a7 * 0.07 + hum_suelo_7a28 * 0.21 + hum_suelo_28a100 * 0.82', {
    'hum_suelo_0a7' : ERA5.select('volumetric_soil_water_layer_1'),
    'hum_suelo_7a28' : ERA5.select('volumetric_soil_water_layer_2'),
    'hum_suelo_28a100' : ERA5.select('volumetric_soil_water_layer_3')
})

#### Visualizar la humedad de suelo de 0-100 cm de prof.

In [ ]:
#Definimos una paleta de colores para el mapa
vis_params = {'min': 0, 'max': 0.4, 'palette': ['4d1103','992206','e43309','e4a20c','c8c8c8','c1ff89','5bff18','379b0e','13580b']}

#Añadimos la capa de visualización a GEE
mapa.addLayer(hum_suelo_0a100, vis_params, 'Humedad del suelo de 0 - 100 cm de prof. (%)')
mapa

Map(bottom=10374.0, center=[-38.169114135560854, -61.36962890625001], controls=(ZoomControl(options=['position…

Exportamos a nuestro Drive

In [ ]:
exportar_mapa = ee.batch.Export.image.toDrive(
  image= hum_suelo_0a100,   #modificar de acuerdo a la layer y AOI que querramos exportar
  description= 'Humedad subsuperficial del suelo',
  scale= 10,
  region= aoi.geometry(),
  maxPixels= 1e13,
  crs='EPSG:32720')

exportar_mapa.start()

In [ ]:
print('Task Status:', exportar_mapa.status()['state']) # Chequeamos el estado de la exportacion al drive